# Approximate q-learning (5 pts)

In this notebook you will teach a __pytorch__ neural network to do Q-learning.

In [32]:
import os
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY")) == 0:
    !bash ../xvfb start
    os.environ['DISPLAY'] = ':1'

In [33]:
import gym
from segwey import segwey
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [34]:
# env = gym.make("CartPole-v0").env
# s = env.reset()
# n_actions = env.action_space.n
# state_dim = env.observation_space.shape
# print(type(n_actions), type(state_dim))
# print(s)
env = segwey()
env.set_param(discrete_level = 10, max_u = 100)
print("discrete_level", env.discrete_level)
print("max_u", env.max_u)
s = env.reset()
n_actions = env.n_actions
state_dim = env.state_dim
print(s)

print((n_actions), (state_dim))

discrete_level 10
max_u 100
[0.         0.42317371 0.         0.        ]
21 (4,)


# Approximate Q-learning: building the network

To train a neural network policy one must have a neural network policy. Let's build it.


Since we're working with a pre-extracted features (cart positions, angles and velocities), we don't need a complicated network yet. In fact, let's build something like this for starters:

![img](https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/yet_another_week/_resource/qlearning_scheme.png)

For your first run, please only use linear layers (nn.Linear) and activations. Stuff like batch normalization or dropout may ruin everything if used haphazardly. 

Also please avoid using nonlinearities like sigmoid & tanh: agent's observations are not normalized so sigmoids may become saturated from init.

Ideally you should start small with maybe 1-2 hidden layers with < 200 neurons and then increase network size if agent doesn't beat the target score.

In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [36]:
network = nn.Sequential()

network.add_module('layer1', nn.Linear(state_dim[0], 64))
network.add_module('relu1', nn.ReLU())
network.add_module('layer2', nn.Linear(64, 64))
network.add_module('relu2', nn.ReLU())
network.add_module('layer3', nn.Linear(64, n_actions))

# hint: use state_dim[0] as input size

In [37]:
state = env.reset()
state = torch.tensor(state[None], dtype=torch.float32)
network(state)

tensor([[-0.0793,  0.0924, -0.1140, -0.1441,  0.0311, -0.0774, -0.0285, -0.0880,
         -0.0052,  0.0240,  0.0569,  0.0442, -0.0308,  0.1552,  0.1170,  0.0107,
          0.0685,  0.0959,  0.0325,  0.0903, -0.0145]],
       grad_fn=<AddmmBackward>)

In [38]:
def get_action(state, epsilon=0):
    """
    sample actions with epsilon-greedy policy
    recap: with p = epsilon pick random action, else pick action with highest Q(s,a)
    """
    state = torch.tensor(state[None], dtype=torch.float32)
    q_values = network(state).detach().numpy()
    
    action = np.argmax(q_values)
    if np.random.binomial(1,p=epsilon):
        action = np.random.choice(range(q_values.shape[-1]))

    return int( action )

In [39]:
c = env.theta_threshold_radians
def castom_reward(next_s, r):
    hor  = max(0, 1 - abs(next_s[0]))
    vert = max(0, 1 - c*next_s[1]**2)
    return r
#     return r*0.85 + vert*0.05 + hor *0.1

In [40]:
s = env.reset()
assert tuple(network(torch.tensor([s]*3, dtype=torch.float32)).size()) == (
    3, n_actions), "please make sure your model maps state s -> [Q(s,a0), ..., Q(s, a_last)]"
assert isinstance(list(network.modules(
))[-1], nn.Linear), "please make sure you predict q-values without nonlinearity (ignore if you know what you're doing)"
assert isinstance(get_action(
    s), int), "get_action(s) must return int, not %s. try int(action)" % (type(get_action(s)))

# test epsilon-greedy exploration
for eps in [0., 0.1, 0.5, 1.0]:
    state_frequencies = np.bincount(
        [get_action(s, epsilon=eps) for i in range(10000)], minlength=n_actions)
    best_action = state_frequencies.argmax()
    assert abs(state_frequencies[best_action] -
               10000 * (1 - eps + eps / n_actions)) < 200
    for other_action in range(n_actions):
        if other_action != best_action:
            assert abs(state_frequencies[other_action] -
                       10000 * (eps / n_actions)) < 200
    print('e=%.1f tests passed' % eps)

e=0.0 tests passed
e=0.1 tests passed
e=0.5 tests passed
e=1.0 tests passed


### Q-learning via gradient descent

We shall now train our agent's Q-function by minimizing the TD loss:
$$ L = { 1 \over N} \sum_i (Q_{\theta}(s,a) - [r(s,a) + \gamma \cdot max_{a'} Q_{-}(s', a')]) ^2 $$


Where
* $s, a, r, s'$ are current state, action, reward and next state respectively
* $\gamma$ is a discount factor defined two cells above.

The tricky part is with  $Q_{-}(s',a')$. From an engineering standpoint, it's the same as $Q_{\theta}$ - the output of your neural network policy. However, when doing gradient descent, __we won't propagate gradients through it__ to make training more stable (see lectures).

To do so, we shall use `x.detach()` function which basically says "consider this thing constant when doingbackprop".

In [41]:
def compute_td_loss(states, actions, rewards, next_states, is_done, gamma=0.99, check_shapes=False):
    """ Compute td loss using torch operations only. Use the formula above. """
    states = torch.tensor(
        states, dtype=torch.float32)    # shape: [batch_size, state_size]
    actions = torch.tensor(actions, dtype=torch.long)    # shape: [batch_size]
    rewards = torch.tensor(rewards, dtype=torch.float32)  # shape: [batch_size]
    # shape: [batch_size, state_size]
    next_states = torch.tensor(next_states, dtype=torch.float32)
    is_done = torch.tensor(is_done, dtype=torch.uint8)  # shape: [batch_size]

    # get q-values for all actions in current states
    predicted_qvalues = network(states)

    # select q-values for chosen actions
    predicted_qvalues_for_actions = predicted_qvalues[
      range(states.shape[0]), actions
    ]

    # compute q-values for all actions in next states
    predicted_next_qvalues = network(next_states)

    # compute V*(next_states) using predicted next q-values
    next_state_values =  torch.max(predicted_next_qvalues, dim=-1)[0]
    assert next_state_values.dtype == torch.float32

    # compute "target q-values" for loss - it's what's inside square parentheses in the above formula.
    target_qvalues_for_actions =  rewards + gamma*next_state_values

    # at the last state we shall use simplified formula: Q(s,a) = r(s,a) since s' doesn't exist
    target_qvalues_for_actions = torch.where(
        is_done, rewards, target_qvalues_for_actions)

    # mean squared error loss to minimize
    loss = torch.mean((predicted_qvalues_for_actions -
                       target_qvalues_for_actions.detach()) ** 2)

    if check_shapes:
        assert predicted_next_qvalues.data.dim(
        ) == 2, "make sure you predicted q-values for all actions in next state"
        assert next_state_values.data.dim(
        ) == 1, "make sure you computed V(s') as maximum over just the actions axis and not all axes"
        assert target_qvalues_for_actions.data.dim(
        ) == 1, "there's something wrong with target q-values, they must be a vector"

    return loss

In [42]:
# sanity checks
s = env.reset()
a = get_action(s)
next_s, r, done, _ = env.step(a)
# s = torch.tensor(s[None], dtype=torch.float32)
# print(network(torch.tensor(s)))
loss = compute_td_loss([s], [a], [r], [next_s], [done], check_shapes=True)
loss.backward()

assert len(loss.size()) == 0, "you must return scalar loss - mean over batch"
assert np.any(next(network.parameters()).grad.detach().numpy() !=
              0), "loss must be differentiable w.r.t. network weights"

### Playing the game

In [43]:
opt = torch.optim.Adam(network.parameters(), lr=1e-4)
epsilon = 0.2

In [44]:
def generate_session(t_max=1000, epsilon=0, train=False):
    """play env with approximate q-learning agent and train it at the same time"""
    total_reward = 0
    s = env.reset()

    for t in range(t_max):
        a = get_action(s, epsilon=epsilon)
        next_s, r, done, _ = env.step(a)
#         r = castom_reward(next_s, r)
        if train:
            opt.zero_grad()
            compute_td_loss([s], [a], [r], [next_s], [done]).backward()
            opt.step()

        total_reward += r
        s = next_s
        if done:
            break

    return total_reward

In [45]:
for i in range(1000):
    session_rewards = [generate_session(
        epsilon=epsilon, train=True) for _ in range(100)]
    print("epoch #{}\tmean reward = {:.3f}\tepsilon = {:.3f}".format(
        i, np.mean(session_rewards), epsilon))

    epsilon *= 0.99
    assert epsilon >= 1e-4, "Make sure epsilon is always nonzero during training"

    if np.mean(session_rewards) > 1000:
        print("You Win!")
        break

epoch #0	mean reward = 6.490	epsilon = 0.200
epoch #1	mean reward = 5.840	epsilon = 0.198
epoch #2	mean reward = 9.620	epsilon = 0.196
epoch #3	mean reward = 16.110	epsilon = 0.194
epoch #4	mean reward = 19.660	epsilon = 0.192
epoch #5	mean reward = 20.460	epsilon = 0.190
epoch #6	mean reward = 19.390	epsilon = 0.188
epoch #7	mean reward = 17.180	epsilon = 0.186
epoch #8	mean reward = 16.520	epsilon = 0.185
epoch #9	mean reward = 17.160	epsilon = 0.183
epoch #10	mean reward = 17.050	epsilon = 0.181
epoch #11	mean reward = 22.190	epsilon = 0.179
epoch #12	mean reward = 20.110	epsilon = 0.177
epoch #13	mean reward = 21.570	epsilon = 0.176
epoch #14	mean reward = 25.520	epsilon = 0.174
epoch #15	mean reward = 22.630	epsilon = 0.172
epoch #16	mean reward = 25.790	epsilon = 0.170
epoch #17	mean reward = 27.430	epsilon = 0.169
epoch #18	mean reward = 23.010	epsilon = 0.167
epoch #19	mean reward = 24.130	epsilon = 0.165
epoch #20	mean reward = 23.970	epsilon = 0.164
epoch #21	mean reward = 23

KeyboardInterrupt: 

In [ ]:
s = env.reset()
print(env.max_u)
X=[]
for t in range(500):
    a = get_action(s)
    s, r, done, _ = env.step(a)
    X.append(s)
    if done:
        break

plt.plot(range(len(X)),np.array(X)[:,[0,1]])

In [ ]:
# torch.save(network.state_dict(), "segwey_network_mod_r.pt")

In [ ]:
for param in network.state_dict():
    print(param, '\t', network.state_dict()[param])

In [ ]:
for param in opt.state_dict():
    print(param, '\t', opt.state_dict()[param])